In [1]:
import numpy as np
import copy
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
import torch
import torch.nn as nn
import dtnnlib as dtnn
# import resnet_cifar

from torchvision import datasets, transforms as T
from torch.utils import data

In [3]:
from tqdm import tqdm
import os, time, sys
import json

In [4]:
mnist_transform = T.Compose([
    T.ToTensor(),
    T.Normalize(
        mean=[0.5,],
        std=[0.5,],
    ),
])

train_dataset = datasets.FashionMNIST(root="../../../../_Datasets/", train=True, download=True, transform=mnist_transform)
test_dataset = datasets.FashionMNIST(root="../../../../_Datasets/", train=False, download=True, transform=mnist_transform)

In [5]:
# train_dataset.data = train_dataset.data.view(-1, 28*28)
# test_dataset.data = test_dataset.data.view(-1, 28*28)

In [6]:
batch_size = 50
train_loader = data.DataLoader(dataset=train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(dataset=test_dataset, num_workers=4, batch_size=batch_size, shuffle=False)

In [7]:
device = torch.device("cuda:0")

In [8]:
criterion = nn.CrossEntropyLoss()

In [9]:
for xx, yy in train_loader:
    xx, yy = xx.to(device), yy.to(device)
    print(xx.shape, yy.shape)
    break

/home/tsuman/All_Files/Program_Files/miniconda/lib/python3.9/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


torch.Size([50, 1, 28, 28]) torch.Size([50])


## Any function as metric

In [10]:
class FunctionDT(nn.Module):
    
    def __init__(self, input_dim, num_centers, func, inv_temp=0.):
        '''
        func [input_dim -> 1]
        '''
        super().__init__()
        self.input_dim = input_dim
        self.num_centers = num_centers
        self.func = func
        
        self.inv_temp = nn.Parameter(torch.ones(1)*inv_temp)
        
        self.centers = torch.randn(num_centers, input_dim)/3.
        self.centers = nn.Parameter(self.centers)
    
    def forward(self, x):
        z = x.unsqueeze(1) - self.centers.unsqueeze(0)
        dists = self.func(z).squeeze(-1)
        dists = -dists*torch.exp(self.inv_temp)
        return dists

In [11]:
from classes import DistanceRegressor, ConvexNN
from nflib.flows import SequentialFlow, ActNorm
import nflib.res_flow as irf

/home/tsuman/All_Files/Program_Files/miniconda/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Merge all models into single and benchmark

In [12]:
def get_models(h = 5, device='cpu'):
    I = 784
    layer1 = {
        "l_0.5":dtnn.DistanceTransform(I, h, p=0.5, bias=False),
        "l_1":dtnn.DistanceTransform(I, h, p=1, bias=False),
        "l_2":dtnn.DistanceTransform(I, h, bias=False),
        "l_inf":dtnn.DistanceTransform(I, h, p=20, bias=False),
        "stereo":dtnn.StereographicTransform(I, h, bias=False),
        "linear":nn.Linear(I, h, bias=False)
    }
    net_dict = {}
    for key in layer1:
        net = nn.Sequential(
            layer1[key],
            nn.BatchNorm1d(h),
            nn.ELU(),
            nn.Linear(h, 10),
            )
        net_dict[key] = net.to(device)
    return net_dict

In [13]:
def get_models_func(h = 500, func_h=500, device='cpu'):
    layer1 = {
        "convex":ConvexNN([784, func_h, 1]),
        "invex":nn.Sequential(
                    ActNorm(784),
                    irf.ResidualFlow(784, [func_h], activation=irf.LeakyReLU),
                    ActNorm(784),
                    DistanceRegressor(784),
                    ),
        "ordinary":nn.Sequential(
                    ActNorm(784),
                    irf.ResidualFlow(784, [func_h], activation=irf.LeakyReLU),
                    ActNorm(784),
                    DistanceRegressor(784),
                    ),
    }
    irf.remove_spectral_norm_model(layer1["ordinary"])

    net_dict = {}
    for key in layer1:
        net = nn.Sequential(
            FunctionDT(784, h, layer1[key]),
            nn.BatchNorm1d(h),
            nn.ELU(),
            nn.Linear(h, 10),
            )
        net_dict[key] = net.to(device)
    return net_dict

In [14]:
## Following is copied from 
### https://github.com/kuangliu/pytorch-cifar/blob/master/main.py

# Training
def train(epoch, model, optimizer):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(tqdm(train_loader)):
        inputs, targets = inputs.to(device).view(-1, 28*28), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    print(f"[Train] {epoch} Loss: {train_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f} {correct}/{total}")
    return

In [15]:
best_acc = -1
def test(epoch, model, model_name):
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(tqdm(test_loader)):
            inputs, targets = inputs.to(device).view(-1, 28*28), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
    print(f"[Test] {epoch} Loss: {test_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f} {correct}/{total}")
    
    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'model': model.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('models'):
            os.mkdir('models')
        torch.save(state, f'./models/{model_name}.pth')
        best_acc = acc

In [16]:
learning_rate = 0.0001
EPOCHS = 50

In [17]:
exp_acc_vals = {}

In [18]:
data_file = "./outputs/00.2_exp_acc_dict.json"

with open(data_file, 'r') as f:
    exp_acc_vals = json.load(f)

In [19]:
exp_acc_vals

{'5': {'l_0.5': 67.05,
  'l_1': 70.7,
  'l_2': 78.15,
  'l_inf': 79.52,
  'stereo': 82.19,
  'linear': 82.74,
  'convex': 79.49,
  'invex': 88.26,
  'ordinary': 83.55},
 '10': {'l_0.5': 72.08,
  'l_1': 77.91,
  'l_2': 82.35,
  'l_inf': 83.98,
  'stereo': 84.73,
  'linear': 84.89,
  'convex': 78.99,
  'invex': 88.41,
  'ordinary': 81.69},
 '20': {'l_0.5': 75.54,
  'l_1': 80.08,
  'l_2': 83.82,
  'l_inf': 85.17,
  'stereo': 85.86,
  'linear': 86.27,
  'convex': 81.03}}

In [20]:
H = [5, 10, 20, 100, 500]
net_keys = {**get_models(1), **get_models_func(1)}.keys()
net_keys

Yes Linear(in_features=784, out_features=500, bias=True)
Success
Yes Linear(in_features=500, out_features=784, bias=True)
Success


dict_keys(['l_0.5', 'l_1', 'l_2', 'l_inf', 'stereo', 'linear', 'convex', 'invex', 'ordinary'])

In [21]:
# H_ = [h for h in H if str(h) not in exp_acc_vals.keys()]
H_ = []
for h in H:
    if str(h) in exp_acc_vals.keys():
        NET_KEYS = [k for k in net_keys if k not in list(exp_acc_vals[str(h)].keys())]
        print(h, NET_KEYS)
        if len(NET_KEYS) > 0:
            H_.append(h)
    else:
        H_.append(h)

H_

5 []
10 []
20 ['invex', 'ordinary']


[20, 100, 500]

In [22]:
# h = 20
# NET_KEYS = list(net_keys)
# if str(h) in exp_acc_vals.keys():
#     NET_KEYS = [k for k in net_keys if k not in list(exp_acc_vals[str(h)].keys())[:-2]]

In [23]:
# NET_KEYS

In [24]:
# str(h) in exp_acc_vals.keys()

In [ ]:
for h in H_:
    net_dict = {**get_models(h), **get_models_func(h)}
    acc_dict = {}
    NET_KEYS = list(net_keys)
    
    if str(h) in exp_acc_vals.keys():
        acc_dict = exp_acc_vals[str(h)]
        NET_KEYS = [k for k in net_keys if k not in list(exp_acc_vals[str(h)].keys())]
        pass
                    
    for key in NET_KEYS:
        print("_________________________")
        print(f"Experimenting for {key} ; h:{h}")
        net = net_dict[key].to(device)
        
        model_name = f"00.2_fmnist_{key}_h{h}"
        
        optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
        best_acc = -1
        for epoch in range(EPOCHS):
            train(epoch, net, optimizer)
            test(epoch, net, model_name)
            scheduler.step()
        acc_dict[key] = float(best_acc)
        exp_acc_vals[str(h)] = acc_dict
        
        ## Save it in the file.
        with open(data_file, "w") as f:
            json.dump(exp_acc_vals, f, indent=3)
        
        pass
        

Yes Linear(in_features=784, out_features=500, bias=True)
Success
Yes Linear(in_features=500, out_features=784, bias=True)
Success
_________________________
Experimenting for invex ; h:20


100%|█████████████████████████████████████████████████| 1200/1200 [00:23<00:00, 52.03it/s]


[Train] 0 Loss: 1.770 | Acc: 40.348 24209/60000


100%|██████████████████████████████████████████████████| 200/200 [00:01<00:00, 135.77it/s]


[Test] 0 Loss: 1.345 | Acc: 58.690 5869/10000
Saving..


...

100%|█████████████████████████████████████████████████| 1200/1200 [00:22<00:00, 53.75it/s]


[Train] 4 Loss: 0.594 | Acc: 80.893 48536/60000


100%|██████████████████████████████████████████████████| 200/200 [00:01<00:00, 135.73it/s]


[Test] 4 Loss: 0.731 | Acc: 75.570 7557/10000


 51%|█████████████████████████▎                        | 607/1200 [00:11<00:11, 53.58it/s]

In [ ]:
exp_acc_vals

In [ ]:
xx

In [ ]:
# net[0].centers.shape

In [ ]:
exp_acc_vals = \
{'5': {'l_0.5': 67.05,
  'l_1': 70.7,
  'l_2': 78.15,
  'l_inf': 79.52,
  'stereo': 82.19,
  'linear': 82.74,
  'convex': 79.49,
  'invex': 88.26,
  'ordinary': 83.55},
 '10': {'l_0.5': 72.08,
  'l_1': 77.91,
  'l_2': 82.35,
  'l_inf': 83.98,
  'stereo': 84.73,
  'linear': 84.89,
  'convex': 78.99,
  'invex': 88.41,
  'ordinary': 81.69}}

In [ ]:
with open()

In [ ]:
data_file = "./outputs/00.2_exp_acc_dict.json"
with open(data_file, "w") as f:
    json.dump(exp_acc_vals, f, indent=3)

In [ ]:
# Opening JSON file
with open(data_file, 'r') as f:
    exp_acc_vals = json.load(f)

In [ ]:
exp_acc_vals